In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------------- CONFIGURATION ----------------
BATCH_SIZE = 15
CONTEXT_WINDOW = 50
MAX_TRAIN_STEPS = 10000
EVAL_INTERVAL = 500
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_SAMPLES = 200
EMBED_DIM = 150
NUM_HEADS = 4
NUM_LAYERS = 3
DROPOUT = 0.2
# ------------------------------------------------

torch.manual_seed(42)

# ---------------- DATA LOADING ----------------
def load_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

def create_vocab(text):
    chars = sorted(list(set(text)))
    return {ch: i for i, ch in enumerate(chars)}, {i: ch for i, ch in enumerate(chars)}
!wget https://www.gutenberg.org/cache/epub/11/pg11.txt
text_data = load_text('pg11.txt')
stoi, itos = create_vocab(text_data)
vocab_size = len(stoi)

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text_data), dtype=torch.long)
split_idx = int(0.9 * len(data))
train_data, val_data = data[:split_idx], data[split_idx:]
# ------------------------------------------------

# ---------------- BATCH FUNCTION ----------------
def create_batch(split):
    dataset = train_data if split == 'train' else val_data
    positions = torch.randint(len(dataset) - CONTEXT_WINDOW, (BATCH_SIZE,))
    x = torch.stack([dataset[i:i+CONTEXT_WINDOW] for i in positions])
    y = torch.stack([dataset[i+1:i+CONTEXT_WINDOW+1] for i in positions])
    return x.to(DEVICE), y.to(DEVICE)

@torch.no_grad()
def compute_loss(model):
    model.eval()
    losses = {}
    for split in ['train', 'val']:
        split_losses = torch.zeros(EVAL_SAMPLES)
        for k in range(EVAL_SAMPLES):
            xb, yb = create_batch(split)
            _, loss = model(xb, yb)
            split_losses[k] = loss.item()
        losses[split] = split_losses.mean()
    model.train()
    return losses
# ------------------------------------------------

# ---------------- TRANSFORMER BLOCK ----------------
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.register_buffer('mask', torch.tril(torch.ones(CONTEXT_WINDOW, CONTEXT_WINDOW)))
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        B, T, C = x.shape
        k, q = self.key(x), self.query(x)
        scores = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5)
        scores = scores.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)
        v = self.value(x)
        return weights @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, EMBED_DIM)
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class FeedForward(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(emb_dim, 4 * emb_dim),
            nn.ReLU(),
            nn.Linear(4 * emb_dim, emb_dim),
            nn.Dropout(DROPOUT),
        )

    def forward(self, x):
        return self.net(x)

class TransformerBlock(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        head_size = emb_dim // num_heads
        self.attn = MultiHeadAttention(num_heads, head_size)
        self.ff = FeedForward(emb_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

# ---------------- GPT MODEL ----------------
class MiniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, EMBED_DIM)
        self.pos_emb = nn.Embedding(CONTEXT_WINDOW, EMBED_DIM)
        self.blocks = nn.Sequential(*[TransformerBlock(EMBED_DIM, NUM_HEADS) for _ in range(NUM_LAYERS)])
        self.norm = nn.LayerNorm(EMBED_DIM)
        self.output_head = nn.Linear(EMBED_DIM, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=DEVICE))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.norm(x)
        logits = self.output_head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, start, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = start[:, -CONTEXT_WINDOW:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1)
            start = torch.cat((start, next_idx), dim=1)
        return start
# ------------------------------------------------

# ---------------- TRAINING ----------------
model = MiniGPT().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for step in range(MAX_TRAIN_STEPS):
    if step % EVAL_INTERVAL == 0 or step == MAX_TRAIN_STEPS - 1:
        losses = compute_loss(model)
        print(f"Step {step}: Train Loss {losses['train']:.4f}, Val Loss {losses['val']:.4f}")

    xb, yb = create_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# ---------------- TEXT GENERATION ----------------
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
generated = model.generate(context, 500)
print(decode(generated[0].tolist()))


--2025-08-02 18:13:54--  https://www.gutenberg.org/cache/epub/11/pg11.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174355 (170K) [text/plain]
Saving to: ‘pg11.txt’

pg11.txt            100%[===================>] 170.27K   691KB/s    in 0.2s    

2025-08-02 18:13:55 (691 KB/s) - ‘pg11.txt’ saved [174355/174355]

Step 0: Train Loss 4.5798, Val Loss 4.5881
Step 500: Train Loss 2.7232, Val Loss 3.0774
Step 1000: Train Loss 2.5311, Val Loss 2.9788
Step 1500: Train Loss 2.4512, Val Loss 2.9504
Step 2000: Train Loss 2.3804, Val Loss 2.9303
Step 2500: Train Loss 2.3328, Val Loss 2.9348
Step 3000: Train Loss 2.3019, Val Loss 2.8831
Step 3500: Train Loss 2.2784, Val Loss 2.8128
Step 4000: Train Loss 2.2174, Val Loss 2.8981
Step 4500: Train Loss 2.1267, Val Loss 2.7745
Step 4999: Train Loss 2.1645, Val 